<a href="https://colab.research.google.com/github/nhanhd06/assignment_3/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
# Get Pip Package to install extrac package 
! pip install kaggle
! pip install datasketch
! pip install lightfm
! pip install faiss-cpu
! pip install annoy
! pip install nmslib
! pip install tensorflow==1.14.0

     |████████████████████████████████| 109.3 MB 25 kB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 488 kB 54.6 MB/s 
     |████████████████████████████████| 3.1 MB 42.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you

In [60]:
# Begin to import component and package 
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import io as io
import re as expr
import faiss
import annoy
import pickle
import nmslib
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from datasketch import MinHash, MinHashLSHForest
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Take the downloadable link from google drive which install kaggle.json to allow dataset from kaggle
file_share_link="https://drive.google.com/file/d/1M4yqyHPmR-EVXzyqeL1dVixkwuEhKpKD/view?usp=sharing"
file_id = file_share_link.split("/")
file_address = 'https://docs.google.com/uc?export=download&id=' + file_id[5]

In [5]:
# Make folder and download that kaggle.json and download nips-papers dataset
! mkdir .kaggle
! wget -O kaggle.json "$file_address" 
! mv kaggle.json ~/.kaggle/ 
! chmod 600 /root/.kaggle/kaggle.json
! kaggle datasets download benhamner/nips-papers
! unzip -j nips-papers.zip
! ls -ahl

mkdir: cannot create directory ‘.kaggle’: File exists
--2021-11-21 01:23:23--  https://docs.google.com/uc?export=download&id=1M4yqyHPmR-EVXzyqeL1dVixkwuEhKpKD
Resolving docs.google.com (docs.google.com)... 142.250.73.238, 2607:f8b0:4004:82a::200e
Connecting to docs.google.com (docs.google.com)|142.250.73.238|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/oq5jkmh0u15r0pq669v55o0a7jg1kv10/1637457750000/15672033725657311753/*/1M4yqyHPmR-EVXzyqeL1dVixkwuEhKpKD?e=download [following]
--2021-11-21 01:23:23--  https://doc-0o-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/oq5jkmh0u15r0pq669v55o0a7jg1kv10/1637457750000/15672033725657311753/*/1M4yqyHPmR-EVXzyqeL1dVixkwuEhKpKD?e=download
Resolving doc-0o-5c-docs.googleusercontent.com (doc-0o-5c-docs.googleusercontent.com)... 172.217.1.193, 2607:f8b0:4004:801::2001
Connecting to doc-0

In [6]:
# Read paper.csv and output type of each column
db = pd.read_csv('papers.csv')
db.dtypes

id             int64
year           int64
title         object
event_type    object
pdf_name      object
abstract      object
paper_text    object
dtype: object

In [7]:
# Sample out one data set and it looks like this dataset contains the entire content of each book as well
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame(db.iloc[0]) 
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


0
id          1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [8]:
# Begin to create a new field by combined title and abstract field and output some samples
db['text'] = db['title'] + ' ' + db['abstract']
df = pd.DataFrame(db['text'].head()) 
df

,text
0,Self-Organization of Associative Database and Its Applications Abstract Missing
1,A Mean Field Theory of Layer IV of Visual Cortex and Its Application to Artificial Neural Networks Abstract Missing
2,Storing Covariance by the Associative Long-Term Potentiation and Depression of Synaptic Strengths in the Hippocampus Abstract Missing
3,Bayesian Query Construction for Neural Network Models Abstract Missing
4,"Neural Network Ensembles, Cross Validation, and Active Learning Abstract Missing"


In [ ]:
# ==>> Locality-sensitive hashing Method

In [10]:
# Method to split a string of test into a set of individual shingles based on whitespace of one string line
def split_data(text):
    # Split and divide each field based on whitespace and make all of them into lower case
    text = expr.sub(r'[^\w\s]', '', text)
    
    tokens = text.lower()
    tokens = tokens.split()
    
    return tokens

In [14]:
# Method to create minhash set using input string data set
def create_min_hash(data, perms):
    
    # Created empty minhash array
    new_min_hash = []
    
    # Loop through the column - text of dataset
    for text in data['text']:

        # Begin to split each text line into each separate shingle
        shingles = split_data(text)

        # Set the number of arrangement permutation the set could have
        min_hash = MinHash(num_perm = perms)

        # Tranform shingles array set and convert into min_hash number 0 or 1
        for s in shingles:
            min_hash.update(s.encode('utf8'))

        # Stored each recently transformed min_hash into outside loop array new_min_hash value
        new_min_hash.append(min_hash)

    # Declare Min Hash Matrix Builder with input arrangement number
    forest = MinHashLSHForest(num_perm = perms)
    
    # Input each tranformed new_min_hash array set into this Matrix Builder with auto increment using enumerate function
    for i, m in enumerate(new_min_hash):
        forest.add(i, m)
    
    # Associate index position number into forest - matrix set
    forest.index()
    
    # Return index marix
    return forest

In [57]:
# Method to query the MinHash Forest 
def predict_recommendation(text, database, perms, num_results, forest):
    
    # Split the target book that needs to find into shingle set
    tokens = split_data(text)
    
    # Declare Min Hash Function with desired arrangement permutation
    min_hash = MinHash(num_perm = perms)
    
    # Transform the shingle set into min hash with either 0 or 1 
    for s in tokens:
        min_hash.update(s.encode('utf8'))

    # Begin to query based on that target book with the number of desired closest associated book list
    return_array = np.array(forest.query(min_hash, num_results))

    # If nothing is returned or the current book is not existed in book list, returns None
    if len(return_array) == 0:
        return None
    
    # Begin to compare and take out the title of those book list position 
    result = database.iloc[return_array]['title']
    
    # Return the result data
    return result

In [58]:
# Declared how many arrangement the set could have and number of returned associated book list
permutations = 256
num_recommendations = 11

# Created Min Hash Matrix with Signatures by using create_min_hash function with input database and arrangement number
forest = create_min_hash(db, permutations)

# Copy and paste one sample book which is in the dataset 
title = 'A Mean Field Theory of Layer IV of Visual Cortex and Its Application to Artificial Neural Networks'

# Using predict_recommendation function to find the number of closest associated book list with book title, dataset, number of arrangement, number of book list and Min Hash Matrix
result = predict_recommendation(title, db, permutations, num_recommendations, forest)

# Output the result list 
print("Top Recommended Books For: " + title)
print("===================================")
print(result)

Top Recommended Books For: A Mean Field Theory of Layer IV of Visual Cortex and Its Application to Artificial Neural Networks
1920    Synchronization of neural networks by mutual learning and its application to cryptography                   
1       A Mean Field Theory of Layer IV of Visual Cortex and Its Application to Artificial Neural Networks          
3012    Sequential Adaptation of Radial Basis Function Neural Networks and its Application to Time-series Prediction
293     A Mean Field Algorithm for Bayes Learning in Large Feed-forward Neural Networks                             
6598    End-to-end Differentiable Proving                                                                           
1041    A Sequence Kernel and its Application to Speaker Recognition                                                
2034    Nearest Neighbor Based Feature Selection for Regression and its Application to Neural Activity              
4145    Bayesian Hierarchical Reinforcement Learning   

In [61]:
# ==> Creating Movie Dataset with movie title and vector data

In [114]:
# Feth movie data from LightFM 
movie_db = fetch_movielens()

# Input train and test data
train_set = movie_db['train']
test_set = movie_db['test']

# Create a model and transform features into vectors
model = LightFM(learning_rate=0.05, loss='warp', no_components=128)
model.fit_partial(train_set, item_features=movie_db['item_features'], epochs=20 )
movie_vectors = movie_db['item_features'] * model.item_embeddings

# Write out the file include with movie list along with each movie matrix array  
with open('movies.pi', 'wb') as f:
  pickle.dump({"name": movie_db['item_labels'], "vector": movie_vectors}, f)

# Load that same file in and use it as data 
with open('movies.pi', 'rb') as f:
  db = pickle.load(f)

In [124]:
# Look at the content of file 
db

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[ 0.06790917,  0.14905521, -0.03146441, ...,  0.23280904,
          0.04859752, -0.47116724],
        [-0.02486507,  0.39040127, -0.12403372, ...,  0.03810447,
         -0.02402395,  0.09534134],
        [ 0.32808727, -0.1561944 , -0.26341832, ...,  0.19235352,
          0.06226182, -0.26510906],
        ...,
        [ 0.15639833,  0.08411242, -0.06831767, ..., -0.33117083,
         -0.18216059,  0.2698071 ],
        [-0.0202959 ,  0.00289584, -0.06635826, ..., -0.12908971,
          0.03184647,  0.12258187],
        [ 0.25045726,  0.05679078, -0.0460018 , ..., -0.09730878,
          0.03169955, -0.01848776]], dtype=float32)}

In [150]:
# ==> Exhaustive Search

In [151]:
class BruteForceIndex():

  # Intialized and set variables - length, vectors, and title 
  def __init__(self, vectors, titles):
    self.length = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.titles = titles

  # Build a brute force index matrix with the size of vectors and add/transform input vector
  def build(self):
    self.index = faiss.IndexFlatL2(self.length)
    self.index.train(self.vectors)
    self.index.add(self.vectors)

  # Input the desired movie name and returns any closed assocciated 10 movie list
  def query(self, vectors, k):
    distances, indices = self.index.search(vectors, k) 
    return [self.titles[i] for i in indices[0]]

In [152]:
# Input vector and title columns in BruteForceIndex and output the result
index = BruteForceIndex(db["vector"], db["name"])
index.build()
movie_vector, movie_name = db['vector'][93:94], db['name'][93]

# Convert list to str and output the result
similar_movies_names = '\n* '.join(index.query(movie_vector, 10))
print("The most similar movies => " + similar_movies_names)

The most similar movies => Home Alone (1990)
* City Slickers II: The Legend of Curly's Gold (1994)
* Three Musketeers, The (1993)
* Santa Clause, The (1994)
* Beverly Hills Cop III (1994)
* Mrs. Doubtfire (1993)
* Robin Hood: Men in Tights (1993)
* Addams Family Values (1993)
* Apple Dumpling Gang, The (1975)
* Ace Ventura: Pet Detective (1994)


In [ ]:
# ==> Product Quantization

In [153]:
class IVPQIndex():

  # Intialized and set variables - length, vectors, and title 
  def __init__(self, vectors, labels):
    self.dimention = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels

  # Build vector matrix using IndexIVFPQ
  def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
    quantizer = faiss.IndexFlatL2(self.dimention)
    self.index = faiss.IndexIVFPQ(quantizer, self.dimention,number_of_partition,search_in_x_partitions,subvector_size)
    self.index.train(self.vectors)
    self.index.add(self.vectors)

  # Use the desired value and search for closest associated movie lists      
  def query(self, vectors, k):
    distances, indices = self.index.search(vectors, k) 
    return [self.labels[i] for i in indices[0]]


In [170]:
# Intialized IVPQIndex Build and return back the associated closest list of input movie 
index = IVPQIndex(db["vector"], db["name"])
index.build()

movie_vector, movie_name = db['vector'][93:94], db['name'][93]
similar_movies_names = '\n* '.join(index.query(movie_vector, 10))
print("The most similar movies => " + similar_movies_names)

The most similar movies => Home Alone (1990)
* City Slickers II: The Legend of Curly's Gold (1994)
* Addams Family Values (1993)
* Beverly Hills Cop III (1994)
* Grease (1978)
* Three Musketeers, The (1993)
* Brady Bunch Movie, The (1995)
* Mrs. Doubtfire (1993)
* With Honors (1994)
* Three Caballeros, The (1945)


In [ ]:
# ==> Trees And Graphs

In [175]:
class AnnoyIndex():

  # Intialized and set variables - length, vectors, and title
  def __init__(self, vectors, labels):
    self.dimention = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels

  # Build a matrix vector using nearest neighbors 
  def build(self, number_of_trees=5):
    self.index = annoy.AnnoyIndex(self.dimention)
    for i, vec in enumerate(self.vectors):
      self.index.add_item(i, vec.tolist())
    self.index.build(number_of_trees)
  
  # Use the desired value and search for closest associated movie lists 
  def query(self, vector, k):
    indices = self.index.get_nns_by_vector(vector.tolist(), k)
    return [self.labels[i] for i in indices]

In [174]:
# Intialized AnnoyIndex Build and return back the associated closest list of input movie 
index = AnnoyIndex(db["vector"], db["name"])
index.build()

movie_vector, movie_name = db['vector'][90], db['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector, 10))
print("The most similar movies => " + similar_movies_names)

The most similar movies => Home Alone (1990)
* City Slickers II: The Legend of Curly's Gold (1994)
* Addams Family Values (1993)
* Beverly Hills Cop III (1994)
* Grease (1978)
* Three Musketeers, The (1993)
* Brady Bunch Movie, The (1995)
* Mrs. Doubtfire (1993)
* With Honors (1994)
* Three Caballeros, The (1945)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


In [ ]:
# ==> Hnsw

In [180]:
class Hnsw_Index():

   # Intialized and set variables - length, vectors, and title
  def __init__(self, vectors, labels):
    self.dimention = vectors.shape[1]
    self.vectors = vectors.astype('float32')
    self.labels = labels

  # Build a matrix vector using nmslib package with hnsw and add vector data in 
  def build(self):
    self.index = nmslib.init(method='hnsw', space='cosinesimil')
    self.index.addDataPointBatch(self.vectors)
    self.index.createIndex({'post': 2})
  
  # Use the desired value and search for closest associated movie lists 
  def query(self, vector, k):
    indices = self.index.knnQuery(vector, k=k)
    return [self.labels[i] for i in indices[0]]

In [181]:
# Intialized Hnsw Function to build and return back the associated closest list of input movie 
index = Hnsw_Index(db["vector"], db["name"])
index.build()
movie_vector, movie_name = db['vector'][90], db['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector, 10))
print("The most similar movies => " + similar_movies_names)

The most similar movies => Home Alone (1990)
* City Slickers II: The Legend of Curly's Gold (1994)
* Addams Family Values (1993)
* Beverly Hills Cop III (1994)
* Grease (1978)
* Three Musketeers, The (1993)
* Brady Bunch Movie, The (1995)
* Mrs. Doubtfire (1993)
* With Honors (1994)
* Three Caballeros, The (1945)
